# Processing the data

In [ ]:
import torch
from transformers import AdamW,AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
batch["labels"] = torch.tensor([1,1])


In [ ]:
optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

## Loading a dataset from the Hub

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

In [ ]:
raw_train_dataset.features

In [ ]:
raw_train_dataset[14]

In [ ]:
raw_train_dataset[87]

## Preprocessing a dataset

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

In [ ]:
inputs = tokenizer(raw_train_dataset[14]['sentence1'], raw_train_dataset[14]['sentence2'])
inputs

In [ ]:
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

## Dynamic padding

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

## SST2

In [ ]:
sst2_datasets = load_dataset("glue", "sst2")

In [ ]:
sst2_datasets

In [ ]:
def tokenize_sst2_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [ ]:
tokenized_sst2_datasets = sst2_datasets.map(tokenize_sst2_function, batched=True)

In [ ]:
sst2_samples = tokenized_sst2_datasets["train"][:8]
sst2_samples = {k: v for k, v in sst2_samples.items() if k not in ["idx", "sentence"]}
[len(x) for x in sst2_samples["input_ids"]]

In [ ]:
sst2_batch = data_collator(sst2_samples)
{k: v.shape for k, v in sst2_batch.items()}

# Fine-tuning a model with the Trainer API

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

## Evaluation

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
preds

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## SST2

In [ ]:
sst2_datasets = load_dataset("glue", "sst2")
sst2_datasets

In [ ]:
def tokenize_sst2(example):
    return tokenizer(example["sentence"], truncation=True)

In [ ]:
tokenized_sst2_datasets = sst2_datasets.map(tokenize_sst2, batched=True)
tokenized_sst2_datasets

In [ ]:
def compute_metrics_sst2(eval_preds):
    metric = evaluate.load("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer-sst2", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_sst2_datasets["train"],
    eval_dataset=tokenized_sst2_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_sst2,
)

In [ ]:
trainer.train()

# A full training with pytorch

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)    

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Prepare for training

In [ ]:
# post=process tokenized datasets
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [ ]:
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

In [ ]:
# define dataloaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
# check dataprocessing
for batch in train_dataloader:
    break
{k:v.shape for k,v in batch.items()}    

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
num_training_steps

## The training loop

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
model.train()

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

In [ ]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

## The evaluation loop

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k:v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## SST2

In [27]:
sst2_dataset = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
def tokenize_sst2(example):
    return tokenizer(example["sentence"], truncation=True)

In [35]:
tokenized_sst2_dataset = sst2_dataset.map(tokenize_sst2, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [36]:
tokenized_sst2_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [38]:
tokenized_sst2_dataset = tokenized_sst2_dataset.remove_columns(["sentence", "idx"])

In [39]:
tokenized_sst2_dataset = tokenized_sst2_dataset.rename_column("label", "labels")

In [41]:
tokenized_sst2_dataset.set_format("torch")
tokenized_sst2_dataset["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [43]:
from torch.utils.data import DataLoader
train_sst2_dataloader = DataLoader(
    tokenized_sst2_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_sst2_dataloader = DataLoader(
    tokenized_sst2_dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [44]:
from transformers import AutoModelForSequenceClassification
model_sst2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
for batch in train_sst2_dataloader:
    break
    
{k:v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 34]),
 'token_type_ids': torch.Size([8, 34]),
 'attention_mask': torch.Size([8, 34])}

In [46]:
outputs = model_sst2(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6060, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [53]:
from torch.optim import AdamW
optimizer_sst2 = AdamW(model_sst2.parameters(), lr=5e-5)
from transformers import get_scheduler

num_epochs = 3
num_training_steps_sst2 = num_epochs * len(train_sst2_dataloader)
lr_scheduler_sst2 = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps_sst2,
)
num_training_steps_sst2

25257

In [54]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_sst2.to(device)
device

device(type='cuda')

In [55]:
from tqdm.auto import tqdm
progress_bar_sst2 = tqdm(range(num_training_steps_sst2))

  0%|          | 0/25257 [00:00<?, ?it/s]

In [57]:
for epoch in range(num_epochs):
    for batch in train_sst2_dataloader:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model_sst2(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer_sst2.step()
        lr_scheduler_sst2.step()
        optimizer_sst2.zero_grad()
        progress_bar_sst2.update(1)

In [58]:
import evaluate
metric_sst2 = evaluate.load("glue", "sst2")
model_sst2.eval()
for batch in eval_sst2_dataloader:
    batch = {k:v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_sst2(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric_sst2.add_batch(predictions=predictions, references=batch["labels"])

metric_sst2.compute()

{'accuracy': 0.8727064220183486}